In [1]:
import arcpy
from timeit import default_timer as timer

print("Start Operations")
start = timer()

#parameters to set
gdb_working = r"D:\ArcGIS_Local\LiDAR\Contours.gdb"
fd_contour_inputs = fr"{gdb_working}\ContourInputs"
fd_contour_outputs = fr"{gdb_working}\ContourYears"
fc_watershed = "cHUC12_Enriched"
fc_grid = "cUSNG_1000m"
contour_years = ["2001", "2007", "2011", "2017", "2019", "2020"]

arcpy.env.parallelProcessingFactor = "100%"
arcpy.env.overwriteOutput = True

for contour_year in contour_years:
    print(f"\nBeginning Routine for Contour Year {contour_year}")
    start = timer()

    # variables
    fc_contours = f"Contours_{contour_year}"

    # load featureclasses into memory
    print("\tLoading Watersheds and USNG into Memory")
    arcpy.env.workspace = fd_contour_inputs
    mem_watersheds = arcpy.conversion.FeatureClassToFeatureClass(
        fc_watershed, "memory", "mem_Watershed"
    )
    mem_grid = arcpy.conversion.FeatureClassToFeatureClass(fc_grid, "memory", "mem_Grid")
    print("\tLoading Contours into Memory")
    mem_contours = arcpy.conversion.FeatureClassToFeatureClass(
        fc_contours, "memory", f"mem_{fc_contours}"
    )

    # set env to memory
    arcpy.env.workspace = r"in_memory"
    arcpy.env.parallelProcessingFactor = "100%"
    arcpy.env.overwriteOutput = True

    # run intersect
    print("\tBeginning Intersect Routine")
    mem_multi_intersect01 = arcpy.analysis.Intersect(
        [mem_contours, mem_watersheds], "mem_multi_intersect01", output_type="POINT"
    )
    print("\t\tIntersect 1 Complete - Contours Intersected Against Watersheds")
    mem_multi_intersect02 = arcpy.analysis.Intersect(
        [mem_contours, mem_grid], "mem_multi_intersect02", output_type="POINT"
    )
    print("\t\tIntersect 2 Complete - Contours Intersected Against USNG Grid")

    print("\tAppending Intersects to single featureclass")
    arcpy.management.Append(
        inputs=mem_multi_intersect02, target=mem_multi_intersect01, schema_type="NO_TEST"
    )
    print("\tConverting multi-point to single-point featureclass")
    mem_point_intersects = arcpy.management.MultipartToSinglepart(
        mem_multi_intersect01, f"mem_point_intersects"
    )
    print("\tSplitting Contours at Intersected Points")
    mem_contours_split = arcpy.management.SplitLineAtPoint(
        mem_contours, mem_point_intersects, f"mem_split_contours", "10 Feet"
    )
    print("\tSpatially Joining Contours for Enrichment")
    print("\t\tJoining Contours Against Watersheds")
    mem_contours_sj_water = arcpy.analysis.SpatialJoin(target_features=mem_contours_split, join_features=mem_watersheds, out_feature_class="mem_contours_sj_water", join_operation="JOIN_ONE_TO_MANY", join_type="KEEP_ALL", match_option="HAVE_THEIR_CENTER_IN")
    print("\t\tJoining Contours Against USNG")
    mem_contours_sj_grid = arcpy.analysis.SpatialJoin(target_features=mem_contours_sj_water, join_features=mem_grid, out_feature_class="mem_contours_sj_grid", join_operation="JOIN_ONE_TO_MANY", join_type="KEEP_ALL", match_option="HAVE_THEIR_CENTER_IN")
    print("\tSorting Data")
    mem_sorted = arcpy.management.Sort(mem_contours_sj_grid, f"mem_Contours_Final", "USNG ASCENDING;HUC12 ASCENDING;Contour ASCENDING;Shape_Length ASCENDING", "LL")
    print("\tDeleting Fields")
    arcpy.management.DeleteField(mem_sorted, "Join_Count;TARGET_FID;JOIN_FID;Join_Count_1;TARGET_FID_1;JOIN_FID_1;ORIG_FID;ORIG_FID_1;ORIG_SEQ;created_user;created_date;last_edited_user;last_edited_date;Shape_Length_1", "DELETE_FIELDS")
    print("\tExporting to FGDB")
    arcpy.conversion.FeatureClassToFeatureClass(mem_sorted, fd_contour_outputs, f"Contours_{contour_year}_Final")
    end = timer()
    print(f"\tRoutine for Contour Year {contour_year} in {(((end - start)/60)/60):.2f} hours")
    arcpy.Delete_management("memory")


print("All Operations Complete")

Start Operations

Beginning Routine for Contour Year 2020
	Loading Watersheds and USNG into Memory
	Loading Contours into Memory
	Beginning Intersect Routine
		Intersect 1 Complete - Contours Intersected Against Watersheds
		Intersect 2 Complete - Contours Intersected Against USNG Grid
	Appending Intersects to single featureclass
	Converting multi-point to single-point featureclass
	Splitting Contours at Intersected Points
	Spatially Joining Contours for Enrichment
		Joining Contours Against Watersheds
		Joining Contours Against USNG
	Sorting Data
	Deleting Fields
	Exporting to FGDB
	Routine for Contour Year 2020 in 187.75 hours
All Operations Complete


### Data Prerequisites
- US National Grid Data can be downloaded from the [US National Grid Information Center](https://usngcenter.org/portfolio-item/usng-gis-data/)
    - alternatively it can be sourced from the [Esri Living Atlas](https://livingatlas.arcgis.com/en/browse/?q=usng#d=2&q=USNG)
- Hydrological Units Data can be downloaded from the [USDA](https://gdg.sc.egov.usda.gov/GDGOrder.aspx?order=QuickState) or this [USGS REST SERVICE](https://hydro.nationalmap.gov/arcgis/rest/services/wbd/MapServer)
    - alternatively the [Esri Hydro team](https://www.arcgis.com/home/user.html?user=esri_hydro) curates a collection as well
- A Contour layer derived from a Digital Surface Model, or the precursor data to create one

### Preparing Your Workspace and Parameter Setting
The staging area for this notebook is a file geodatabase, it will hold the inputs and outputs, the rest of the processing is done in the ArcPy `"memory"` environment. The following parameters may be set:
```py
#parameters to set
gdb_working = r"D:\ArcGIS_Local\LiDAR\Contours.gdb"
fd_contour_inputs = fr"{gdb_working}\ContourInputs"
fd_contour_outputs = fr"{gdb_working}\ContourYears"
fc_watershed = "cHUC12_Enriched"
fc_grid = "cUSNG_1000m"
contour_years = ["2001", "2007", "2011", "2015", "2019", "2020"]
```
- `gdb_working` - a file geodatabase
- `fd_contour_inputs`, - a feature dataset within the file geodatabase
- `fd_contour_outputs` - a feature dataset within the file geodatabase
- `fc_watershed` - a feature class containing a Hydrological Units feature class
    - this should be the longest digit feature class available (i.e. HUC12)
    - this feature class may be additionally enriched to contain data relevant to the shorter digit features (i.e. HUC10, HUC8, etc.)
- `fc_grid` - a feature class containing the US National Grid at the desired scale
- `contour_years` - a list corresponding to the number of Contour feature classes within the `fd_contour_inputs` feature dataset
    - the naming convention used is `"Contours_"` with the year immediately following like: `"Contours_2022"`

### The Workflow
This notebook is fairly straight-forward; using a loop it proceeds through a series of geoprocessing tasks within the ArcPy `"memory"` environment. The processing is memory-intensive but is significantly faster than performed using traditional writes on the file geodatabase.
- converts the enrichment data `fc_watershed`, `fc_grid` to memory feature classes, and the `contour` data to a memory feature class as well
- intersects the `contour` memory feature class against both the enrichment data feature classes in turn
    - creates a *multi-point geometry* output for `fc_watershed` as `mem_multi_intersect01`
    - creates a *multi-point geometry* output for `fc_grid` as `mem_multi_intersect02`
- the *multi-point* geometry output above is consolidated into a single memory feature class through an append method into `mem_multi_intersect01` as the target feature class
- the *multi-point* geometry is converted into *point geometry* within `mem_point_intersects`
- `mem_point_intersects` is then used to split the `contour` data within `mem_contours_split`
    - this makes sure that contours are split at each USNG tile and again at any water boundary (sub-watershed, watershed, sub-basin, basin, etc.)
    - this steps ensures that the spatial join in the next step is effective at capturing attribution in the appropriate granularity
- the newly split contours are then spatially joined against `fc_watershed` in `mem_contours_sj_water`
- `mem_contours_sj_water` is then spatially joined against `fc_grid` in `mem_contours_sj_grid`
- then sort the Contour geometry by: grid, hydrological unit, contour elevation, shape
    - this forces the drawing per grid/hydro unit area providing better performance when zoomed in
- for a clean-up operation, delete fields removes all the extra fields created during processing
- finally export the feature class from the `"memory"` environment back to the file geodatabse
- the loop deletes all `"memory"` features to re-allocate memory and initiates another cycle of the loop based on the input list

### Additional Notes
This process is both time and memory intensive; the resultant dataset is significantly larger than the input but is more suitable for viewing at larger scales. Processing a modern, LiDAR-derived DSM with nominal pulse spacing at 0.5 Meters, or 4 points per meter; for 25 square miles with a machine consistently utilizing 10-15GB of RAM (64GB RAM installed):
- approximately 12-16 hours for 1 meter contours
- approximately 190 hours for 1 foot contours